In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [19]:
def ScheduleCrawling():
    
    driver = webdriver.Chrome('/Users/dongyokim/main/jupyter/chromedriver/chromedriver')
    driver.implicitly_wait(2)

    driver.get("https://www.airport.co.kr/gimpo/cms/frCon/index.do?MENU_ID=1060")
    time.sleep(1)
    
    
    final_code_list = []
    final_code_list2 = []

    input_year = input('몇 년도의 운항 스케줄을 뽑으시겠습니까? ')

    if input_year == '2018':
        year_len = 4
    elif input_year == '2019':
        year_len = 3
    elif input_year == '2020':
        year_len = 2
    elif input_year == '2021':
        year_len = 1
    elif input_year == '2022':
        year_len = 0
    else:
        print('연도를 잘못 입력했습니다. ')
        return False
    
    if year_len == 0:
        month_len = 12
    else:
        month_len = 12
    driver.find_element(By.CSS_SELECTOR, '#sendForm > fieldset > div:nth-child(1) > span.inpGrp > span > label').click()

    # 제주 선택
    driver.find_element(By.CSS_SELECTOR, '#sendForm > fieldset > div:nth-child(3) > select > option:nth-child(7)').click()

    time.sleep(0.5)
    driver.find_element(By.CSS_SELECTOR, '#fd-but-datepicker1').click()
    time.sleep(0.5)


    for j in range(year_len):
        driver.find_element(By.CSS_SELECTOR, '#datepicker1-prev-year-but').click()

    for page in range(11):
        driver.find_element(By.CSS_SELECTOR, '#datepicker1-prev-month-but').click()

    print('################ 항공 스케줄 크롤링을 시작합니다. ##########################')
    time.sleep(1.0)
    # 12월까지 
    for num in range(month_len):
        print(f'{input_year}년 {num+1}월의 항공 스케줄 데이터를 추출하고 있습니다.')
        time.sleep(0.5)
        # 2, 3, 4 주의 첫날만 클릭하자
        for i in range(2, 5):
            driver.find_element(By.CSS_SELECTOR, '#fd-datepicker1 > div > table > tbody > tr:nth-child({}) > td:nth-child(1)'.format(i)).click()
            time.sleep(1.0)    

            driver.find_element(By.CSS_SELECTOR, '#sendForm > fieldset > button').click()
            time.sleep(0.5)



            # 출발기준 
            time.sleep(2)
            code_list = driver.find_elements(By.CSS_SELECTOR, '#outTbody > tr')

            print('#######{}년 {}월 {}주차 데이터셋 추출 시작(출발기준) #######'.format(input_year, num+1, i))
            for code in code_list:
                # 항공편
                air_num = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[1].split(')')[0]
                
                
                airline_name = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[0]

                start_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[0]    

                end_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[1]    

            
            
                start_time =  code.find_element(By.CSS_SELECTOR, 'td.start > p').text

                end_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(3) > p').text

                # 운항 기간 
                run_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(11) > p').text.split('~')[0].rstrip()

            
                print([air_num, airline_name,start_loc, end_loc, start_time, end_time, run_time])
                print()
                final_code_list.append([air_num, airline_name ,start_loc, end_loc, start_time, end_time, run_time])



            # 도착 기준 
            code_list2 = driver.find_elements(By.CSS_SELECTOR, '#inTbody > tr')
            print('#######{}년 {}월 {}주차 데이터셋 추출 시작(도착 기준) #######'.format(input_year, num+1, i))

            for code in code_list2:
                # 항공편
                air_num = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[1].split(')')[0]
                
                
                airline_name = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[0]
                
                start_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[0]    

                end_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[1]    


                start_time =  code.find_element(By.CSS_SELECTOR, 'td.start > p').text

                end_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(3) > p').text

                # 운항 기간 
                run_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(11) > p').text.split('~')[0].rstrip()

              
                print([air_num, airline_name, start_loc, end_loc, start_time, end_time, run_time])
                print()
                final_code_list2.append([air_num, airline_name, start_loc, end_loc, start_time, end_time, run_time])

            time.sleep(0.5)
            driver.find_element(By.CSS_SELECTOR, '#fd-but-datepicker1').click()
            print('-------------------------------------------')
            time.sleep(0.5)

        driver.find_element(By.CSS_SELECTOR, '#datepicker1-next-month-but').click()

        #fd-datepicker1 > div > table > tbody > tr:nth-child(2)
    print('############## {}년 {}월 까지의 운항 스케줄 크롤링을 종료합니다. #####################'.format(input_year, num+1))
    driver.quit()
    
    # 출발 기준
    final_code_df = pd.DataFrame(final_code_list, columns=['운항편명', '항공사명', '출발공항', '도착공항', '출발예정시간', '도착예정시간', '운항일자'])
    
    # 도착기준 
    final_code_df2 = pd.DataFrame(final_code_list2, columns=['운항편명', '항공사명', '출발공항', '도착공항', '출발예정시간', '도착예정시간', '운항일자'])
    
    # 중복 제거
    final_code_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
    
    
    # 운항일자순으로 정렬
    final_code_df.sort_values(by='운항일자', inplace=True)
    
    
    # 중복 제거
    final_code_df2.drop_duplicates(keep='first', ignore_index=True, inplace=True)
    
    # 운항일자순으로 정렬
    final_code_df2.sort_values(by='운항일자', inplace=True)
    
    
    
    
    return final_code_df, final_code_df2, input_year

        
        

In [20]:
# 위에 함수 안에 크롬 드라이버 경로 자기 경로로 바꾸주자
final_code_df, final_code_df2, input_year = ScheduleCrawling()

몇 년도의 운항 스케줄을 뽑으시겠습니까? 2018
################ 항공 스케줄 크롤링을 시작합니다. ##########################
2018년 6월의 항공 스케줄 데이터를 추출하고 있습니다.
#######2018년 6월 2주차 데이터셋 추출 시작(출발기준) #######
['LJ301', '진에어', '서울/김포', '제주', '06:05', '07:20', '2018.03.25']

['OZ8901', '아시아나항공', '서울/김포', '제주', '06:05', '07:15', '2018.03.25']

['LJ303', '진에어', '서울/김포', '제주', '06:10', '07:25', '2018.04.02']

['ZE201', '이스타항공', '서울/김포', '제주', '06:10', '07:20', '2018.03.25']

['7C151', '제주항공', '서울/김포', '제주', '06:15', '07:30', '2018.03.25']

['ZE203', '이스타항공', '서울/김포', '제주', '06:15', '07:25', '2018.03.25']

['TW701', '티웨이항공', '서울/김포', '제주', '06:20', '07:30', '2018.03.25']

['TW701', '티웨이항공', '서울/김포', '제주', '06:20', '07:30', '2018.04.06']

['7C101', '제주항공', '서울/김포', '제주', '06:25', '07:35', '2018.03.25']

['OZ8903', '아시아나항공', '서울/김포', '제주', '06:25', '07:35', '2018.03.25']

['TW701', '티웨이항공', '서울/김포', '제주', '06:40', '07:50', '2018.03.25']

['KE1201', '대한항공', '서울/김포', '제주', '06:45', '07:55', '2018.05.01']

['ZE205', '이스타항공', '서울/김포', '

['LJ317', '진에어', '서울/김포', '제주', '15:55', '17:05', '2018.04.16']

['ZE223', '이스타항공', '서울/김포', '제주', '15:55', '17:05', '2018.03.25']

['KE1245', '대한항공', '서울/김포', '제주', '16:00', '17:10', '2018.05.29']

['KE1245', '대한항공', '서울/김포', '제주', '16:00', '17:10', '2018.06.01']

['7C125', '제주항공', '서울/김포', '제주', '16:05', '17:15', '2018.03.25']

['OZ8971', '아시아나항공', '서울/김포', '제주', '16:05', '17:10', '2018.03.25']

['KE1243', '대한항공', '서울/김포', '제주', '16:20', '17:30', '2018.03.30']

['ZE225', '이스타항공', '서울/김포', '제주', '16:35', '17:45', '2018.03.25']

['KE1247', '대한항공', '서울/김포', '제주', '16:40', '17:50', '2018.03.25']

['7C145', '제주항공', '서울/김포', '제주', '16:45', '17:55', '2018.03.25']

['LJ331', '진에어', '서울/김포', '제주', '17:00', '18:05', '2018.03.25']

['KE1249', '대한항공', '서울/김포', '제주', '17:10', '18:20', '2018.04.03']

['LJ329', '진에어', '서울/김포', '제주', '17:15', '18:25', '2018.03.25']

['LJ331', '진에어', '서울/김포', '제주', '17:20', '18:30', '2018.03.25']

['LJ331', '진에어', '서울/김포', '제주', '17:20', '18:30', '2018.05.28']

['LJ3

['LJ318', '진에어', '제주', '서울/김포', '12:55', '14:05', '2018.03.25']

['OZ8408', '아시아나항공', '제주', '서울/김포', '13:00', '14:05', '2018.03.25']

['ZE214', '이스타항공', '제주', '서울/김포', '13:00', '14:05', '2018.03.25']

['7C116', '제주항공', '제주', '서울/김포', '13:05', '14:15', '2018.03.25']

['BX8028', '에어부산', '제주', '서울/김포', '13:05', '14:10', '2018.03.25']

['KE1216', '대한항공', '제주', '서울/김포', '13:05', '14:15', '2018.06.01']

['OZ8932', '아시아나항공', '제주', '서울/김포', '13:10', '14:20', '2018.03.25']

['KE1222', '대한항공', '제주', '서울/김포', '13:15', '14:25', '2018.03.26']

['KE1224', '대한항공', '제주', '서울/김포', '13:30', '14:40', '2018.04.01']

['KE1224', '대한항공', '제주', '서울/김포', '13:30', '14:40', '2018.04.04']

['TW720', '티웨이항공', '제주', '서울/김포', '13:30', '14:40', '2018.04.05']

['ZE216', '이스타항공', '제주', '서울/김포', '13:30', '14:40', '2018.03.25']

['KE1224', '대한항공', '제주', '서울/김포', '13:35', '14:45', '2018.03.26']

['KE1224', '대한항공', '제주', '서울/김포', '13:35', '14:45', '2018.03.30']

['KE1226', '대한항공', '제주', '서울/김포', '13:35', '14:45', '2018.03.

['BX8011', '에어부산', '서울/김포', '제주', '06:55', '08:05', '2018.03.25']

['7C103', '제주항공', '서울/김포', '제주', '07:00', '08:10', '2018.03.25']

['TW703', '티웨이항공', '서울/김포', '제주', '07:00', '08:10', '2018.03.25']

['OZ8911', '아시아나항공', '서울/김포', '제주', '07:10', '08:20', '2018.04.11']

['BX8013', '에어부산', '서울/김포', '제주', '07:20', '08:30', '2018.03.25']

['KE1203', '대한항공', '서울/김포', '제주', '07:20', '08:30', '2018.06.04']

['KE1203', '대한항공', '서울/김포', '제주', '07:20', '08:30', '2018.06.07']

['OZ8913', '아시아나항공', '서울/김포', '제주', '07:25', '08:35', '2018.03.25']

['BX8027', '에어부산', '서울/김포', '제주', '07:45', '08:55', '2018.03.25']

['BX8027', '에어부산', '서울/김포', '제주', '07:45', '08:55', '2018.05.03']

['7C105', '제주항공', '서울/김포', '제주', '07:50', '09:00', '2018.03.25']

['LJ305', '진에어', '서울/김포', '제주', '07:50', '09:05', '2018.06.07']

['OZ8915', '아시아나항공', '서울/김포', '제주', '07:55', '09:05', '2018.03.25']

['OZ8915', '아시아나항공', '서울/김포', '제주', '07:55', '09:05', '2018.06.04']

['KE1205', '대한항공', '서울/김포', '제주', '08:00', '09:10', '2018.

['LJ331', '진에어', '서울/김포', '제주', '17:20', '18:30', '2018.05.28']

['LJ331', '진에어', '서울/김포', '제주', '17:20', '18:25', '2018.06.01']

['TW727', '티웨이항공', '서울/김포', '제주', '17:25', '18:35', '2018.03.25']

['TW729', '티웨이항공', '서울/김포', '제주', '17:40', '18:50', '2018.03.25']

['KE1251', '대한항공', '서울/김포', '제주', '17:50', '19:00', '2018.03.25']

['OZ8973', '아시아나항공', '서울/김포', '제주', '17:50', '18:55', '2018.03.25']

['OZ8411', '아시아나항공', '서울/김포', '제주', '17:55', '19:00', '2018.03.25']

['BX8015', '에어부산', '서울/김포', '제주', '18:10', '19:20', '2018.03.25']

['TW731', '티웨이항공', '서울/김포', '제주', '18:10', '19:20', '2018.03.25']

['ZE227', '이스타항공', '서울/김포', '제주', '18:10', '19:20', '2018.03.25']

['7C127', '제주항공', '서울/김포', '제주', '18:15', '19:25', '2018.03.25']

['OZ8981', '아시아나항공', '서울/김포', '제주', '18:15', '19:25', '2018.04.11']

['BX8023', '에어부산', '서울/김포', '제주', '18:20', '19:30', '2018.03.25']

['KE1253', '대한항공', '서울/김포', '제주', '18:20', '19:30', '2018.03.26']

['KE1253', '대한항공', '서울/김포', '제주', '18:20', '19:30', '2018.06.

...

['OZ8963', '아시아나항공', '서울/김포', '제주', '14:05', '15:15', '2018.10.28']

['TW719', '티웨이항공', '서울/김포', '제주', '14:05', '15:15', '2018.10.28']

['LJ317', '진에어', '서울/김포', '제주', '14:20', '15:35', '2018.10.28']

['TW721', '티웨이항공', '서울/김포', '제주', '14:20', '15:30', '2018.10.28']

['KE1223', '대한항공', '서울/김포', '제주', '14:25', '15:35', '2018.12.04']

['7C121', '제주항공', '서울/김포', '제주', '14:40', '15:50', '2018.10.28']

['BX8021', '에어부산', '서울/김포', '제주', '14:45', '15:55', '2018.10.28']

['OZ8965', '아시아나항공', '서울/김포', '제주', '14:45', '15:55', '2018.10.28']

['ZE217', '이스타항공', '서울/김포', '제주', '14:45', '15:55', '2018.10.28']

['OZ8967', '아시아나항공', '서울/김포', '제주', '14:50', '16:00', '2018.10.28']

['7C155', '제주항공', '서울/김포', '제주', '15:00', '16:10', '2018.10.28']

['OZ8971', '아시아나항공', '서울/김포', '제주', '15:00', '16:10', '2018.10.28']

['TW723', '티웨이항공', '서울/김포', '제주', '15:15', '16:25', '2018.10.28']

['OZ8973', '아시아나항공', '서울/김포', '제주', '15:25', '16:30', '2018.10.28']

['OZ8975', '아시아나항공', '서울/김포', '제주', '15:30', '16:40', '2

['KE1210', '대한항공', '제주', '서울/김포', '12:10', '13:20', '2018.12.04']

['KE1210', '대한항공', '제주', '서울/김포', '12:10', '13:20', '2018.12.07']

['TW714', '티웨이항공', '제주', '서울/김포', '12:15', '13:25', '2018.10.28']

['7C154', '제주항공', '제주', '서울/김포', '12:20', '13:30', '2018.10.28']

['LJ314', '진에어', '제주', '서울/김포', '12:30', '13:45', '2018.10.28']

['TW716', '티웨이항공', '제주', '서울/김포', '12:30', '13:40', '2018.10.28']

['ZE254', '이스타항공', '제주', '서울/김포', '12:40', '13:50', '2018.10.28']

['KE1212', '대한항공', '제주', '서울/김포', '12:50', '14:00', '2018.10.29']

['OZ8940', '아시아나항공', '제주', '서울/김포', '12:50', '14:00', '2018.10.28']

['BX8028', '에어부산', '제주', '서울/김포', '12:55', '14:10', '2018.10.28']

['OZ8944', '아시아나항공', '제주', '서울/김포', '13:00', '14:05', '2018.10.28']

['ZE212', '이스타항공', '제주', '서울/김포', '13:00', '14:10', '2018.10.28']

['KE1214', '대한항공', '제주', '서울/김포', '13:15', '14:25', '2018.10.28']

['KE1214', '대한항공', '제주', '서울/김포', '13:15', '14:25', '2018.10.29']

['TW718', '티웨이항공', '제주', '서울/김포', '13:30', '14:40', '2018.10.

['7C151', '제주항공', '서울/김포', '제주', '06:45', '07:55', '2018.10.28']

['KE1201', '대한항공', '서울/김포', '제주', '06:45', '07:55', '2018.10.28']

['ZE203', '이스타항공', '서울/김포', '제주', '06:45', '07:55', '2018.10.28']

['OZ8907', '아시아나항공', '서울/김포', '제주', '06:50', '08:00', '2018.10.28']

['BX8011', '에어부산', '서울/김포', '제주', '06:55', '08:05', '2018.10.28']

['7C103', '제주항공', '서울/김포', '제주', '07:00', '08:10', '2018.10.28']

['TW703', '티웨이항공', '서울/김포', '제주', '07:00', '08:10', '2018.10.28']

['OZ8911', '아시아나항공', '서울/김포', '제주', '07:10', '08:20', '2018.10.28']

['BX8013', '에어부산', '서울/김포', '제주', '07:20', '08:30', '2018.10.28']

['OZ8913', '아시아나항공', '서울/김포', '제주', '07:35', '08:45', '2018.10.28']

['ZE205', '이스타항공', '서울/김포', '제주', '07:40', '08:50', '2018.10.28']

['BX8027', '에어부산', '서울/김포', '제주', '07:45', '08:55', '2018.10.28']

['7C105', '제주항공', '서울/김포', '제주', '07:50', '09:00', '2018.10.28']

['KE1203', '대한항공', '서울/김포', '제주', '08:00', '09:10', '2018.10.28']

['TW705', '티웨이항공', '서울/김포', '제주', '08:15', '09:25', '2018.1

['BX8015', '에어부산', '서울/김포', '제주', '18:10', '19:20', '2018.10.28']

['OZ8413', '아시아나항공', '서울/김포', '제주', '18:10', '19:15', '2018.10.28']

['7C167', '제주항공', '서울/김포', '제주', '18:15', '19:20', '2018.12.04']

['LJ325', '진에어', '서울/김포', '제주', '18:15', '19:30', '2018.10.28']

['7C127', '제주항공', '서울/김포', '제주', '18:20', '19:30', '2018.10.28']

['BX8023', '에어부산', '서울/김포', '제주', '18:20', '19:25', '2018.10.28']

['ZE223', '이스타항공', '서울/김포', '제주', '18:20', '19:30', '2018.10.28']

['7C167', '제주항공', '서울/김포', '제주', '18:30', '19:40', '2018.12.04']

['OZ8983', '아시아나항공', '서울/김포', '제주', '18:35', '19:45', '2018.10.28']

['KE1231', '대한항공', '서울/김포', '제주', '18:40', '19:50', '2018.12.10']

['KE1231', '대한항공', '서울/김포', '제주', '18:40', '19:50', '2018.12.12']

['OZ8985', '아시아나항공', '서울/김포', '제주', '18:45', '19:55', '2018.10.28']

['TW731', '티웨이항공', '서울/김포', '제주', '18:45', '19:55', '2018.10.28']

['KE1233', '대한항공', '서울/김포', '제주', '19:00', '20:10', '2018.11.05']

['KE1233', '대한항공', '서울/김포', '제주', '19:00', '20:10', '2018.11.

['OZ8958', '아시아나항공', '제주', '서울/김포', '15:40', '16:50', '2018.10.28']

['LJ320', '진에어', '제주', '서울/김포', '15:50', '17:05', '2018.10.28']

['TW722', '티웨이항공', '제주', '서울/김포', '15:50', '17:00', '2018.10.28']

['TW722', '티웨이항공', '제주', '서울/김포', '15:50', '17:00', '2018.10.28']

['OZ8962', '아시아나항공', '제주', '서울/김포', '16:05', '17:15', '2018.10.28']

['TW724', '티웨이항공', '제주', '서울/김포', '16:15', '17:25', '2018.10.28']

['BX8010', '에어부산', '제주', '서울/김포', '16:20', '17:30', '2018.10.28']

['LJ322', '진에어', '제주', '서울/김포', '16:25', '17:40', '2018.10.28']

['OZ8412', '아시아나항공', '제주', '서울/김포', '16:25', '17:35', '2018.10.28']

['7C120', '제주항공', '제주', '서울/김포', '16:30', '17:45', '2018.10.28']

['KE1222', '대한항공', '제주', '서울/김포', '16:30', '17:40', '2018.12.10']

['KE1222', '대한항공', '제주', '서울/김포', '16:30', '17:40', '2018.12.12']

['BX8020', '에어부산', '제주', '서울/김포', '16:35', '17:45', '2018.10.28']

['OZ8412', '아시아나항공', '제주', '서울/김포', '16:35', '17:40', '2018.10.28']

['OZ8964', '아시아나항공', '제주', '서울/김포', '16:40', '17:50', '2018

In [21]:
final_code_df

,운항편명,항공사명,출발공항,도착공항,출발예정시간,도착예정시간,운항일자
0,LJ301,진에어,서울/김포,제주,06:05,07:20,2018.03.25
116,ZE221,이스타항공,서울/김포,제주,15:40,16:50,2018.03.25
115,LJ325,진에어,서울/김포,제주,15:25,16:30,2018.03.25
114,7C155,제주항공,서울/김포,제주,15:25,16:40,2018.03.25
110,OZ8963,아시아나항공,서울/김포,제주,15:05,16:15,2018.03.25
...,...,...,...,...,...,...,...
556,KE1209,대한항공,서울/김포,제주,09:40,10:50,2018.12.05
568,KE1231,대한항공,서울/김포,제주,18:40,19:50,2018.12.10
566,KE1223,대한항공,서울/김포,제주,14:25,15:35,2018.12.11
567,KE1223,대한항공,서울/김포,제주,14:25,15:35,2018.12.12


In [22]:
final_code_df2

,운항편명,항공사명,출발공항,도착공항,출발예정시간,도착예정시간,운항일자
140,7C146,제주항공,제주,서울/김포,18:30,19:40,2018.03.25
97,LJ324,진에어,제주,서울/김포,15:25,16:40,2018.03.25
147,ZE228,이스타항공,제주,서울/김포,19:45,20:55,2018.03.25
148,TW732,티웨이항공,제주,서울/김포,19:55,21:05,2018.03.25
94,TW722,티웨이항공,제주,서울/김포,15:00,16:10,2018.03.25
...,...,...,...,...,...,...,...
558,KE1236,대한항공,제주,서울/김포,20:45,21:55,2018.12.10
556,KE1222,대한항공,제주,서울/김포,16:30,17:40,2018.12.10
555,KE1210,대한항공,제주,서울/김포,12:10,13:20,2018.12.12
557,KE1222,대한항공,제주,서울/김포,16:30,17:40,2018.12.12


In [24]:
# 2018년일 경우 2017년 말 운항일짜가 나올 수도 있음(다른 연도도 해당) 나중에 엑셀에서 전처리 필요 
# -> 만약 2019년 데이터에 운행일자가 2018년도가 나오면 해당 데이터를 2018년 데이터로 이동시키자 


#  엑셀 저장 코드 
final_code_df.to_excel('1. {}년 항공 스케줄 데이터(출발)_중복제거.xlsx'.format(input_year), index=False)
final_code_df2.to_excel('1. {}년 항공 스케줄 데이터(도착)_중복제거.xlsx'.format(input_year), index=False)